# Download Publicly Available Neutrophil Dataset

**Gregory Way, 2018**

Here, I download [GSE103706](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE103706) which is associated with [Rincon et al. 2018](https://doi.org/10.1186/s12864-018-4957-6).

This dataset includes two acute myeloid leukemia (AML) cell lines; PLB-985 and HL-60.
There are 14 samples total in this dataset.
The cell lines are exposed to two treatments - DMSO and DMSO+Nutridoma.
The treatments are demonstrated to induce neutrophil differentiation in these cell lines.

We hypothesized that our constructed feature identified through our interpret compression approach would have higher activation patterns in the cell lines with induced neutrophil differentiation.

In [1]:
import os
import csv
import pandas as pd
from sklearn import preprocessing

from scripts.utils import download_geo

In [2]:
base_url = 'ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE103nnn/GSE103706/suppl/'
name = 'GSE103706_Merged_data_FPKM_and_normalized.xlsx'
directory = 'download'

In [3]:
download_geo(base_url, name, directory)

In [4]:
path = 'download/GSE103706_Merged_data_FPKM_and_normalized.xlsx'
! sha256sum $path

8a56404922fc9307eb15de097867a33d97ed68a400a75ab4cd09d84e67def523  download/GSE103706_Merged_data_FPKM_and_normalized.xlsx


## Process the Data

In [5]:
# Load Data
geo_df = pd.read_excel(path, index_col=0, skiprows=1)

print(geo_df.shape)
geo_df.head(2)

(60707, 18)


,ens_gene_id,ncbi_gene_id,gene_short,symbol,"PLB-985, not differentiated, replicate 1","PLB-985, DMSO, day 6, replicate 1","PLB-985, DMSO+Nutridoma, day 4","PLB-985, DMSO+Nutridoma, day 6, replicate 1","HL-60, not differentiated, replicate 1","HL-60, DMSO, day 6, replicate 1","HL-60, DMSO+Nutridoma, day 6, replicate 1","PLB-985, not differentiated, replicate 2","PLB-985, DMSO, day 6, replicate 2","PLB-985, DMSO+Nutridoma, day 2","PLB-985, DMSO+Nutridoma, day 6, replicate 2","HL-60, not differentiated, replicate 2","HL-60, DMSO, day 6, replicate 2","HL-60, DMSO+Nutridoma, day 6, replicate 2"
tracking_id,,,,,,,,,,,,,,,,,,
ENSG00000000003.14,ENSG00000000003.14,7105,TSPAN6,TSPAN6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000000005.5,ENSG00000000005.5,64102,TNMD,TNMD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Update Gene Names

In [6]:
# Load curated gene names from versioned resource 
commit = '721204091a96e55de6dcad165d6d8265e67e2a48'
url = 'https://raw.githubusercontent.com/cognoma/genes/{}/data/genes.tsv'.format(commit)
gene_df = pd.read_table(url)

# Only consider protein-coding genes
gene_df = (
    gene_df.query("gene_type == 'protein-coding'")
)

symbol_to_entrez = dict(zip(gene_df.symbol,
                            gene_df.entrez_gene_id))

In [7]:
# Add alternative symbols to entrez mapping dictionary
gene_df = gene_df.dropna(axis='rows', subset=['synonyms'])
gene_df.synonyms = gene_df.synonyms.str.split('|')

all_syn = (
    gene_df.apply(lambda x: pd.Series(x.synonyms), axis=1)
    .stack()
    .reset_index(level=1, drop=True)
)

# Name the synonym series and join with rest of genes
all_syn.name = 'all_synonyms'
gene_with_syn_df = gene_df.join(all_syn)

# Remove rows that have redundant symbols in all_synonyms
gene_with_syn_df = (
    gene_with_syn_df
    
    # Drop synonyms that are duplicated - can't be sure of mapping
    .drop_duplicates(['all_synonyms'], keep=False)

    # Drop rows in which the symbol appears in the list of synonyms
    .query('symbol not in all_synonyms')
)

In [8]:
# Create a synonym to entrez mapping and add to dictionary
synonym_to_entrez = dict(zip(gene_with_syn_df.all_synonyms,
                             gene_with_syn_df.entrez_gene_id))

symbol_to_entrez.update(synonym_to_entrez)

In [9]:
# Load gene updater
url = 'https://raw.githubusercontent.com/cognoma/genes/{}/data/updater.tsv'.format(commit)
updater_df = pd.read_table(url)
old_to_new_entrez = dict(zip(updater_df.old_entrez_gene_id,
                             updater_df.new_entrez_gene_id))

In [10]:
# Update the symbol column to entrez_gene_id
geo_df.symbol = geo_df.symbol.replace(symbol_to_entrez)
geo_df = geo_df.loc[geo_df.symbol.isin(symbol_to_entrez.values()), :]
geo_df.symbol = geo_df.symbol.replace(old_to_new_entrez)
geo_df.index = geo_df.symbol
geo_df.index.name = 'entrez_gene_id'
geo_df = geo_df.drop(['ens_gene_id', 'ncbi_gene_id', 'gene_short', 'symbol'], axis='columns')

## Scale Data and Output to File

In [11]:
# Scale RNAseq data using zero-one normalization
geo_scaled_zeroone_df = preprocessing.MinMaxScaler().fit_transform(geo_df.transpose())
geo_scaled_zeroone_df = pd.DataFrame(geo_scaled_zeroone_df,
                                     columns=geo_df.index,
                                     index=geo_df.columns)

os.makedirs('data', exist_ok=True)

file = os.path.join('data', 'GSE103706_processed_matrix.tsv.gz')
geo_scaled_zeroone_df.to_csv(file, sep='\t', compression='gzip')

geo_scaled_zeroone_df.head()

entrez_gene_id,7105,64102,8813,6359,55732,2268,3075,2519,2729,4800,...,79447,140032,101234261,100996720,100527949,100862671,100861540,140,100033411,100129520
"PLB-985, not differentiated, replicate 1",0.0,0.0,0.634427,0.166941,1.000000,0.000000,0.0,0.012190,0.358691,0.403020,...,1.000000,0.0,0.0,0.173369,0.020122,0.025108,0.0,0.000093,0.0,0.000000
"PLB-985, DMSO, day 6, replicate 1",0.0,0.0,0.294013,0.876342,0.041159,0.089441,0.0,0.000000,0.954808,0.656400,...,0.495317,0.0,0.0,1.000000,0.017264,0.499439,0.0,0.352886,0.0,0.000000
"PLB-985, DMSO+Nutridoma, day 4",0.0,0.0,0.441410,0.786486,0.105087,0.391584,0.0,0.000000,0.466314,0.870757,...,0.670646,0.0,1.0,0.739464,0.023452,0.374970,0.0,0.309807,0.0,0.992937
"PLB-985, DMSO+Nutridoma, day 6, replicate 1",0.0,0.0,0.325642,1.000000,0.159371,0.371718,0.0,0.005417,0.267552,0.964469,...,0.536067,0.0,0.0,0.700795,0.095464,0.229591,0.0,0.480849,0.0,1.000000
"HL-60, not differentiated, replicate 1",0.0,0.0,0.425104,0.044019,0.438063,0.208925,0.0,0.451546,0.117225,0.000000,...,0.475571,0.0,0.0,0.142413,0.056802,0.106059,0.0,0.000154,0.0,0.000000
